# Cut video into timestamped chunks

This script takes a full video file and slices it into smaller clips,
based on the timestamps generated from the aligned audio-transcript segments. The output is a series of short MP4 clips
that match the segmented MP3s and transcript CSV timestamped rows.

Useful for turning long video recordings into timestamped snippets.

# Connect to Google Drive

In [1]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install Required Packages

In [2]:
!pip install moviepy
!apt-get install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (54.3 kB/s)
Selecting previously unselected package tree.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...


# Import Libraries

In [3]:
import os
import pandas as pd
from moviepy.editor import VideoFileClip
from tqdm import tqdm

  if event.key is 'enter':



# Explore Data structure

In [5]:
!tree -L 2 '/content/drive/MyDrive/a_tutorial_notebooks/data'

/content/drive/MyDrive/a_tutorial_notebooks/data
└── 1
    ├── 1_splits.csv
    ├── audio_clips
    ├── media
    └── transcription

4 directories, 1 file


#Cut video based on start and end times
Cuts a portion from the input video based on start and end times,
then saves that subclip (including audio) to the output directory.

In [6]:
def cut_video(input_video, output_directory, start_time, end_time, output_filename):
    # Load the full video using MoviePy
    video = VideoFileClip(input_video)

    # Extract the segment between start_time and end_time
    subclip = video.subclip(start_time, end_time)

    # Build the full output path for the new video clip
    chunk_filename = os.path.join(output_directory, output_filename)

    # Export the subclip with audio included using H.264 encoding
    subclip.write_videofile(chunk_filename, codec="libx264")

    # Close readers to free memory and avoid resource warnings
    video.reader.close()
    video.audio.reader.close_proc()

# Cuts clips based on timestamps in CSV
Loops through subfolders and cuts clips based on timestamps defined in a CSV file.

In [9]:
data_root = "/content/drive/MyDrive/a_tutorial_notebooks/data"

def main():
    # Root directory containing all subfolders with video + CSV
    main_folder = data_root

    # List all subfolders (e.g., "0", "1", "2", ...)
    subfolders = [f for f in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, f))]

    # Process each subfolder individually
    for folder in tqdm(subfolders, desc="Processing folders"):
        subfolder_path = os.path.join(main_folder, folder)

        # Find the CSV file ending with "_soundbite_yes.csv"
        csv_file = next(
            (f.path for f in os.scandir(subfolder_path) if f.name.endswith("_splits.csv")),
            None
        )

        # Look inside the 'processed' subfolder for an MP4 video file
        processed_folder = os.path.join(subfolder_path, "media")
        mp4_file = next(
            (f.path for f in os.scandir(processed_folder) if f.name.endswith(".mp4") and not f.name.startswith(".")),
            None
        )

        # Skip this folder if either the CSV or MP4 file is missing
        if csv_file is None or mp4_file is None:
            print(f"CSV or MP4 not found in {folder}. Skipping.")
            continue

        # Create output folder for the cut video chunks
        output_directory = os.path.join(subfolder_path, "video_clips")
        os.makedirs(output_directory, exist_ok=True)

        # Load the CSV containing start/end times and segment labels
        df = pd.read_csv(csv_file)

        # Loop through each row in the CSV and cut the corresponding video segment
        for index, row in df.iterrows():
            start_time = row["start_time"]     # e.g., 12.5 seconds
            end_time = row["end_time"]         # e.g., 15.7 seconds
            segment_name = row["chunk"]     # e.g., "chunk_0"
            output_filename = f"{segment_name}.mp4"

            # Cut the video based on these times and save to output
            cut_video(mp4_file, output_directory, start_time, end_time, output_filename)


main()


Processing folders:   0%|          | 0/1 [00:03<?, ?it/s]

Moviepy - Building video /content/drive/MyDrive/a_tutorial_notebooks/data/1/video_clips/1.mp4.
MoviePy - Writing audio in 1TEMP_MPY_wvf_snd.mp3



Processing folders:   0%|          | 0/1 [00:03<?, ?it/s]

MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/a_tutorial_notebooks/data/1/video_clips/1.mp4




Processing folders:   0%|          | 0/1 [00:06<?, ?it/s]

...

Moviepy - Building video /content/drive/MyDrive/a_tutorial_notebooks/data/1/video_clips/711.mp4.
MoviePy - Writing audio in 711TEMP_MPY_wvf_snd.mp3



Processing folders:   0%|          | 0/1 [20:12<?, ?it/s]

MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/a_tutorial_notebooks/data/1/video_clips/711.mp4




Processing folders: 100%|██████████| 1/1 [20:15<00:00, 1215.32s/it]

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/a_tutorial_notebooks/data/1/video_clips/711.mp4
